# error analysis 
series_idごとのoof scoreをチェック
errorがどういう波形に対して発生しているかを確認する

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
import sys
sys.path.append("/kaggle/src/dss_utils")

from dss_metrics import score

# Config

In [3]:
output_dir = os.path.join("/kaggle", "working", "_oof")
exp_name = "exp009_normal_2input"
# exp_name = "debug"

In [4]:
oof_df = pd.DataFrame()
for i in range(5):
    print("fold", i)
    df = pd.read_parquet(os.path.join(output_dir, exp_name, f"oof_df_fold{i}.parquet"))
    for col in df.columns:
        # 64bit float -> 16bit float
        if df[col].dtype == np.float64:
            df[col] = df[col].astype(np.float16)
        # 64bit int -> 16bit int
        elif df[col].dtype == np.int64:
            df[col] = df[col].astype(np.int16)

    oof_df = pd.concat([oof_df, df], axis=0)

fold 0
fold 1
fold 2
fold 3
fold 4


In [5]:
train_event_df = pd.read_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")

In [6]:
# sub_df = pd.read_csv(os.path.join(output_dir, exp_name, "sub_df.csv"))

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/_oof/exp009_normal_2input/sub_df.csv'

In [ ]:
score_df = pd.read_csv(os.path.join(output_dir, exp_name, "score_df.csv"))
display(score_df)

,series_id,score,detected_event_num
0,038441c925bb,0.631418,264
1,03d92c9f6f8a,0.000000,0
2,0402a003dae9,0.503222,213
3,04f547b8017d,0.000000,0
4,05e1944c3818,0.000000,0
...,...,...,...
264,fa149c3c4bde,0.000000,0
265,fb223ed2278c,0.000000,0
266,fbf33b1a2c10,0.000000,0
267,fcca183903b7,0.000000,0


In [ ]:
# scoreの高い方から順に表示
score_df = score_df.sort_values("score", ascending=False)
for series_id in score_df.head(5)["series_id"]:
    break
    df = oof_df[oof_df["series_id"]==series_id]
    print("score", score_df[score_df["series_id"]==series_id]["score"])
    # plt.figure(figsize=(10, 5))
    # plt.subplot(3, 1, 1)
    # plt.plot(df["anglez"])
    # plt.subplot(3, 1, 2)
    # plt.plot(df["enmo"])
    # plt.subplot(3, 1, 3)
    # plt.plot(df["event"], label="event")
    # plt.plot(df["class_pred"], label="class_pred")
    # plt.legend()
    # plt.show()


In [ ]:
score_df = score_df[score_df["series_id"].isin(oof_df["series_id"].unique())]
display(score_df)

,series_id,score,detected_event_num
184,bb5612895813,0.912185,180
221,d8de352c2657,0.856157,155
114,72ba4a8afff4,0.851263,168
93,5c55a5e717d6,0.817334,200
208,ce9164297046,0.816196,264
232,dff367373725,0.808056,183
193,c38707ef76df,0.806144,310
21,1319a1935f48,0.801524,328
144,8fb18e36697d,0.787695,166
212,d0f613c700f7,0.780506,321


# low score seires_ids

In [ ]:
score_df = score_df.sort_values("score")
count = 0
seires_date_key_list = []
for series_id in score_df.head(100)["series_id"]:
    df = oof_df[oof_df["series_id"]==series_id]
    if len(df) == 0:
        continue
    else:
        count += 1
    event = train_event_df[train_event_df["series_id"]==series_id]
    display(event)
    print("score", score_df[score_df["series_id"]==series_id]["score"])
    # plt.figure(figsize=(10, 5))
    # plt.subplot(3, 1, 1)
    # plt.plot(df["step"], df["anglez"])
    # plt.subplot(3, 1, 2)
    # plt.plot(df["step"], df["enmo"])
    # plt.subplot(3, 1, 3)
    # plt.plot(df["step"], df["event"], label="event")
    # plt.plot(df["step"], df["class_pred"], label="class_pred")
    # plt.legend()
    # plt.show()
    seires_date_key_list.extend(df["series_date_key"].unique())
    if count == 10:
        break

,series_id,night,event,step,timestamp
6923,854206f602d0,1,onset,5016.0,2018-09-21T19:58:00-0400
6924,854206f602d0,1,wakeup,12444.0,2018-09-22T06:17:00-0400
6925,854206f602d0,2,onset,NaN,NaN
6926,854206f602d0,2,wakeup,NaN,NaN
6927,854206f602d0,3,onset,NaN,NaN
...,...,...,...,...,...
6998,854206f602d0,38,wakeup,NaN,NaN
6999,854206f602d0,39,onset,NaN,NaN
7000,854206f602d0,39,wakeup,NaN,NaN
7001,854206f602d0,40,onset,NaN,NaN


score 132    0.146364
Name: score, dtype: float64


,series_id,night,event,step,timestamp
9452,b7fc34995d0f,1,onset,NaN,NaN
9453,b7fc34995d0f,1,wakeup,NaN,NaN
9454,b7fc34995d0f,2,onset,24636.0,2018-06-28T21:58:00-0400
9455,b7fc34995d0f,2,wakeup,31272.0,2018-06-29T07:11:00-0400
9456,b7fc34995d0f,3,onset,43368.0,2018-06-29T23:59:00-0400
...,...,...,...,...,...
9515,b7fc34995d0f,32,wakeup,NaN,NaN
9516,b7fc34995d0f,33,onset,NaN,NaN
9517,b7fc34995d0f,33,wakeup,NaN,NaN
9518,b7fc34995d0f,34,onset,NaN,NaN


score 181    0.208056
Name: score, dtype: float64


,series_id,night,event,step,timestamp
310,062dbd4c95e6,1,onset,7872.0,2018-08-22T23:11:00-0400
311,062dbd4c95e6,1,wakeup,14484.0,2018-08-23T08:22:00-0400
312,062dbd4c95e6,2,onset,NaN,NaN
313,062dbd4c95e6,2,wakeup,NaN,NaN
314,062dbd4c95e6,3,onset,NaN,NaN
...,...,...,...,...,...
393,062dbd4c95e6,42,wakeup,NaN,NaN
394,062dbd4c95e6,43,onset,NaN,NaN
395,062dbd4c95e6,43,wakeup,NaN,NaN
396,062dbd4c95e6,44,onset,NaN,NaN


...

score 210    0.579846
Name: score, dtype: float64


,series_id,night,event,step,timestamp
7454,8b8b9e29171c,1,onset,9396.0,2018-04-26T00:18:00-0400
7455,8b8b9e29171c,1,wakeup,14568.0,2018-04-26T07:29:00-0400
7456,8b8b9e29171c,2,onset,27060.0,2018-04-27T00:50:00-0400
7457,8b8b9e29171c,2,wakeup,32244.0,2018-04-27T08:02:00-0400
7458,8b8b9e29171c,3,onset,44544.0,2018-04-28T01:07:00-0400
...,...,...,...,...,...
7521,8b8b9e29171c,34,wakeup,NaN,NaN
7522,8b8b9e29171c,35,onset,NaN,NaN
7523,8b8b9e29171c,35,wakeup,NaN,NaN
7524,8b8b9e29171c,36,onset,NaN,NaN


score 140    0.585748
Name: score, dtype: float64


,series_id,night,event,step,timestamp
10496,c7b2155a4a47,1,onset,7560.0,2017-08-25T22:15:00-0400
10497,c7b2155a4a47,1,wakeup,10236.0,2017-08-26T01:58:00-0400
10498,c7b2155a4a47,2,onset,20556.0,2017-08-26T16:18:00-0400
10499,c7b2155a4a47,2,wakeup,30000.0,2017-08-27T05:25:00-0400
10500,c7b2155a4a47,3,onset,38076.0,2017-08-27T16:38:00-0400
10501,c7b2155a4a47,3,wakeup,45000.0,2017-08-28T02:15:00-0400
10502,c7b2155a4a47,4,onset,55164.0,2017-08-28T16:22:00-0400
10503,c7b2155a4a47,4,wakeup,64284.0,2017-08-29T05:02:00-0400
10504,c7b2155a4a47,5,onset,71748.0,2017-08-29T15:24:00-0400
10505,c7b2155a4a47,5,wakeup,80568.0,2017-08-30T03:39:00-0400


score 199    0.604926
Name: score, dtype: float64


In [ ]:
key_tmp = None
show_idx = 10
count = 0
show_num = 5
low_score_key_list = []
for idx, key in enumerate(seires_date_key_list):
    if count > show_num:
        break
    df = oof_df[oof_df["series_date_key"]==key]
    series_id = df["series_id"].unique()[0]
    min_step = df["step"].min()
    max_step = df["step"].max()
    event = train_event_df[train_event_df["series_id"]==series_id]
    event = event[(event["step"]>=min_step) & (event["step"]<=max_step)]
    if len(event) == 0:
        continue
    
    sub = sub_df[sub_df["series_id"]==series_id]
    sub = sub[(sub["step"]>=min_step) & (sub["step"]<=max_step)]
    sub = sub.sort_values("score", ascending=True)[:5]
    score = score_df[score_df["series_id"]==series_id]["score"].values[0]
    low_score_key_list.append(key)

    # display(event)
    # display(sub)
    # print("score", score)
    # print("series_date_key", key)
    # plt.figure(figsize=(10, 5))
    # plt.subplot(3, 1, 1)
    # plt.plot(df["step"], df["anglez"])
    # plt.subplot(3, 1, 2)
    # plt.plot(df["step"], df["enmo"])
    # plt.subplot(3, 1, 3)
    # plt.plot(df["step"], df["event"], label="event")
    # plt.plot(df["step"], df["class_pred"], label="class_pred")
    # plt.legend()
    # plt.show()

In [ ]:
low_score_key_df = pd.DataFrame(
    {
        "series_date_key": low_score_key_list
    }
).to_csv(os.path.join(output_dir, exp_name, "low_score_keys.csv"))

In [ ]:
print(low_score_key_df, seires_date_key_list)

None [3713, 3714, 3715, 3716, 3717, 3718, 3719, 3720, 3721, 3722, 3723, 3724, 3725, 3726, 3727, 3728, 3729, 3730, 3731, 3732, 3733, 3734, 3735, 3736, 3737, 3738, 3739, 3740, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749, 3750, 3751, 3752, 5067, 5068, 5069, 5070, 5071, 5072, 5073, 5074, 5075, 5076, 5077, 5078, 5079, 5080, 5081, 5082, 5083, 5084, 5085, 5086, 5087, 5088, 5089, 5090, 5091, 5092, 5093, 5094, 5095, 5096, 5097, 5098, 5099, 5100, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 1109, 1110, 1111, 1112, 1113, 1114, 1115, 1116, 1117, 1118, 1119, 1120, 1121, 1122, 1123, 1124, 1125, 1126, 1127, 1128, 1129, 1130, 1131, 1132, 1133, 1134, 1135, 1136, 1137, 1138, 1139, 1140, 1141, 1142, 1143, 4735, 4736, 47